In [125]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report 
import shap

In [126]:
heart_csv_path = 'C:/Users/Rawan Alamily/Downloads/McSCert Co-op/explainable-ai-heart/predictive-models/heart-attributes-model/data/heart.csv'
dataframe = pd.read_csv(heart_csv_path)
train, val, test = np.split(dataframe.sample(frac=1), [int(0.6*len(dataframe)), int(0.9*len(dataframe))])
# train, test = train_test_split(dataframe, test_size=0.2, random_state=42)
y_train = train.pop('target')
X_train = train
y_val = val.pop('target')
X_val = val
y_test = test.pop('target')
X_test = test
print(len(X_train))
print(len(X_val))
print(len(X_test))

615
307
103


In [127]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)
X_train_NORM = scaler.transform(X_train)
X_val_NORM = scaler.transform(X_val)
X_test_NORM = scaler.transform(X_test)

In [128]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Dense(units=128, activation='relu'), 
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(units=128, activation='relu'), 
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])
model1.compile(optimizer="adam", 
              loss ="binary_crossentropy", 
              metrics=["accuracy"])

In [129]:
model1.fit(X_train_NORM, y_train, 
          epochs=70,
          batch_size=32,
          validation_data=(X_val_NORM, y_val))

Epoch 1/70
20/20 [==============================] - 1s 8ms/step - loss: 0.5878 - accuracy: 0.7106 - val_loss: 0.4451 - val_accuracy: 0.8371
Epoch 2/70
20/20 [==============================] - 0s 3ms/step - loss: 0.4077 - accuracy: 0.8407 - val_loss: 0.3664 - val_accuracy: 0.8371
Epoch 3/70
20/20 [==============================] - 0s 4ms/step - loss: 0.3602 - accuracy: 0.8569 - val_loss: 0.3435 - val_accuracy: 0.8534
Epoch 4/70
20/20 [==============================] - 0s 3ms/step - loss: 0.3416 - accuracy: 0.8667 - val_loss: 0.3270 - val_accuracy: 0.8534
Epoch 5/70
20/20 [==============================] - 0s 3ms/step - loss: 0.3146 - accuracy: 0.8699 - val_loss: 0.3150 - val_accuracy: 0.8664
Epoch 6/70
20/20 [==============================] - 0s 4ms/step - loss: 0.3049 - accuracy: 0.8813 - val_loss: 0.2995 - val_accuracy: 0.8599
Epoch 7/70
20/20 [==============================] - 0s 5ms/step - loss: 0.3004 - accuracy: 0.8748 - val_loss: 0.2877 - val_accuracy: 0.8697
Epoch 8/70
20/20 [==

In [130]:
model1.evaluate(X_test_NORM, y_test) 

4/4 [==============================] - 0s 2ms/step - loss: 0.0492 - accuracy: 0.9903


[0.0492335706949234, 0.9902912378311157]

In [131]:
# fit kernel explainer using training data
explainer = shap.KernelExplainer(model1, X_train_NORM[:500, :])

Using 500 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


In [132]:
# test on various individual samples using test dataset 
shap_values = explainer.shap_values(X_test_NORM[1,:],nsamples='auto')
shap.force_plot(explainer.expected_value, shap_values[0], X_test_NORM[1,:])

In [133]:
i=0
for label in dataframe.columns:
    if label != 'target':
        print(label,";",shap_values[0][i])
        i+=1
print(X_test.iloc[1,:])
print(y_test.iloc[1])

age ; 0.0029664177979439876
sex ; -0.03939419905609909
cp ; -0.17951381253413873
trestbps ; -0.04769081493086205
chol ; 0.04859405025932323
fbs ; -0.007225805380130124
restecg ; 0.015439340991243888
thalach ; -0.003399577424987227
exang ; -0.030124640535806158
oldpeak ; 0.07486439244211872
slope ; 0.0783114946756704
ca ; -0.21833898035868615
thal ; -0.18753927946090698
age          59.0
sex           1.0
cp            0.0
trestbps    140.0
chol        177.0
fbs           0.0
restecg       1.0
thalach     162.0
exang         1.0
oldpeak       0.0
slope         2.0
ca            1.0
thal          3.0
Name: 556, dtype: float64
0


In [134]:
shap_values = explainer.shap_values(X_test_NORM[2,:],nsamples='auto')
shap.force_plot(explainer.expected_value, shap_values[0], X_test_NORM[2,:])

In [135]:
i=0
for label in dataframe.columns:
    if label != 'target':
        print(label,";",shap_values[0][i])
        i+=1
print(X_test.iloc[2,:])
print(y_test.iloc[2])

age ; -0.0011764385612037243
sex ; 0.044683414487060924
cp ; -0.0720048974325268
trestbps ; -0.02262469115627247
chol ; -0.0017162677590534525
fbs ; 0.002913146920418108
restecg ; -0.008365892359602972
thalach ; 0.0012383758136094714
exang ; 0.013819244347091891
oldpeak ; -0.1928425313374557
slope ; -0.055399730670370906
ca ; -0.13658083804934296
thal ; -0.07300546765327454
age          62.0
sex           0.0
cp            0.0
trestbps    160.0
chol        164.0
fbs           0.0
restecg       0.0
thalach     145.0
exang         0.0
oldpeak       6.2
slope         0.0
ca            3.0
thal          3.0
Name: 393, dtype: float64
0


In [136]:
i=0
for label in dataframe.columns:
    if label != 'target':
        print(label,";",shap_values[0][i])
        i+=1
print(X_test.iloc[3,:])
print(y_test.iloc[3])

age ; -0.0011764385612037243
sex ; 0.044683414487060924
cp ; -0.0720048974325268
trestbps ; -0.02262469115627247
chol ; -0.0017162677590534525
fbs ; 0.002913146920418108
restecg ; -0.008365892359602972
thalach ; 0.0012383758136094714
exang ; 0.013819244347091891
oldpeak ; -0.1928425313374557
slope ; -0.055399730670370906
ca ; -0.13658083804934296
thal ; -0.07300546765327454
age          70.0
sex           1.0
cp            0.0
trestbps    145.0
chol        174.0
fbs           0.0
restecg       1.0
thalach     125.0
exang         1.0
oldpeak       2.6
slope         0.0
ca            0.0
thal          3.0
Name: 948, dtype: float64
0


In [137]:
i=0
for label in dataframe.columns:
    if label != 'target':
        print(label,";",shap_values[0][i])
        i+=1
print(X_test.iloc[3,:])
print(y_test.iloc[3])

age ; -0.0011764385612037243
sex ; 0.044683414487060924
cp ; -0.0720048974325268
trestbps ; -0.02262469115627247
chol ; -0.0017162677590534525
fbs ; 0.002913146920418108
restecg ; -0.008365892359602972
thalach ; 0.0012383758136094714
exang ; 0.013819244347091891
oldpeak ; -0.1928425313374557
slope ; -0.055399730670370906
ca ; -0.13658083804934296
thal ; -0.07300546765327454
age          70.0
sex           1.0
cp            0.0
trestbps    145.0
chol        174.0
fbs           0.0
restecg       1.0
thalach     125.0
exang         1.0
oldpeak       2.6
slope         0.0
ca            0.0
thal          3.0
Name: 948, dtype: float64
0


In [139]:
shap_values = explainer.shap_values(X_test_NORM[5,:],nsamples='auto')
shap.force_plot(explainer.expected_value, shap_values[0], X_test_NORM[4,:])

In [140]:
i=0
for label in dataframe.columns:
    if label != 'target':
        print(label,";",shap_values[0][i])
        i+=1
print(X_test.iloc[5,:])
print(y_test.iloc[5])

age ; 0.014114195064013901
sex ; -0.02140645049519463
cp ; 0.017754269250994964
trestbps ; 0.014774652073132888
chol ; 0.05923888696926788
fbs ; -0.004184769280820788
restecg ; 0.011911276532405507
thalach ; 0.07404072017302005
exang ; 0.01150983817876211
oldpeak ; 0.06757880182075465
slope ; 0.05785651150956789
ca ; 0.12683472448280714
thal ; 0.06891471147537231
age          41.0
sex           1.0
cp            1.0
trestbps    120.0
chol        157.0
fbs           0.0
restecg       1.0
thalach     182.0
exang         0.0
oldpeak       0.0
slope         2.0
ca            0.0
thal          2.0
Name: 217, dtype: float64
1
